# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'05-16-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'05-16-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-05-17 04:20:34,33.93911,67.709953,63484,2742,54686.0,6056.0,Afghanistan,163.079130,4.319199
1,NaN,NaN,NaN,Albania,2021-05-17 04:20:34,41.15330,20.168300,132015,2432,124312.0,5271.0,Albania,4587.358399,1.842215
2,NaN,NaN,NaN,Algeria,2021-05-17 04:20:34,28.03390,1.659600,125311,3374,87359.0,34578.0,Algeria,285.765153,2.692501
3,NaN,NaN,NaN,Andorra,2021-05-17 04:20:34,42.50630,1.521800,13510,127,13155.0,228.0,Andorra,17485.277940,0.940044
4,NaN,NaN,NaN,Angola,2021-05-17 04:20:34,-11.20270,17.873900,30637,659,25715.0,4263.0,Angola,93.217155,2.150994


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,61455,61755,61842,62063,62403,62718,63045,63355,63412,63484
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,131577,131666,131723,131753,131803,131845,131890,131939,131978,132015
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,123692,123900,124104,124288,124483,124682,124889,125059,125194,125311


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2673,2683,2686,2698,2710,2713,2721,2730,2733,2742
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2408,2411,2412,2416,2420,2423,2426,2427,2429,2432
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3315,3321,3328,3335,3343,3350,3355,3360,3366,3374


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,54019,54040,54222,54382,54503,54534,54619,54634,54663,54686
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,115253,116126,117089,118041,119061,120072,121122,122105,123081,124312
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,86149,86280,86420,86554,86703,86857,87003,87137,87251,87359


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6918,6918,6920,6920,6926,6928,6938,6971,7001,7005
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21107,21123,21131,21135,21154,21170,21191,21290,21392,21411
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2307,2307,2308,2308,2310,2314,2317,2319,2320,2320


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,108,108,108,108,108,108,108,108,108,108
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,307,308,308,308,308,309,309,309,310,310
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,57,57,57,57,57,56,56,56,56,56


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-05-17 04:20:34,32.539527,-86.644082,7005,108,NaN,NaN,"Autauga, Alabama, US",12538.259142,1.541756
689,1075.0,Lamar,Alabama,US,2021-05-17 04:20:34,33.779950,-88.096680,1433,36,NaN,NaN,"Lamar, Alabama, US",10380.296994,2.512212
690,1077.0,Lauderdale,Alabama,US,2021-05-17 04:20:34,34.901719,-87.656247,9460,243,NaN,NaN,"Lauderdale, Alabama, US",10201.770751,2.568710


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,32940778,585970,0.0
India,24684077,270284,20795335.0
Brazil,15627243,435751,13764520.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,32940778,585970,0.0,32354808.0,2021-05-17 04:20:34,37.950547,-91.419547
India,24684077,270284,20795335.0,3618458.0,2021-05-17 04:20:34,23.088275,81.806127
Brazil,15627243,435751,13764520.0,1426972.0,2021-05-17 04:20:34,-12.669522,-48.480493
France,5939019,107777,378958.0,5452284.0,2021-05-17 04:20:34,6.430808,-34.730285
Turkey,5117374,44760,4947256.0,125358.0,2021-05-17 04:20:34,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3768737,62690,0.0
Texas,2929050,50941,0.0
Florida,2292004,36075,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3768737,62690,0.0,3706047.0,2021-05-17 04:20:34,37.843962,-120.728594
Texas,2929050,50941,0.0,2878109.0,2021-05-17 04:20:34,31.660643,-98.653069
Florida,2292004,36075,0.0,2255929.0,2021-05-17 04:20:34,28.940755,-82.700744
New York,2085477,52953,0.0,2032524.0,2021-05-17 04:20:34,42.544151,-75.474183
Illinois,1366179,24795,0.0,1341384.0,2021-05-17 04:20:34,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1237411,24115,0.0
Illinois,Cook,546908,10144,0.0
Arizona,Maricopa,544676,9950,0.0
Florida,Miami-Dade,494903,6304,0.0
Texas,Harris,396552,6330,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1237411,24115,0.0,1213296.0,2021-05-17 04:20:34,34.308284,-118.228241,6037.0
Illinois,Cook,546908,10144,0.0,536764.0,2021-05-17 04:20:34,41.841448,-87.816588,17031.0
Arizona,Maricopa,544676,9950,0.0,534726.0,2021-05-17 04:20:34,33.348359,-112.491815,4013.0
Florida,Miami-Dade,494903,6304,0.0,488599.0,2021-05-17 04:20:34,25.611236,-80.551706,12086.0
Texas,Harris,396552,6330,0.0,390222.0,2021-05-17 04:20:34,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,63355,131939,125059,13510,30030,1240,3269466,220447,29964,635780,...,4462603,235206,96117,4,212998,3816,303270,6523,92356,38535
2021-05-15,63412,131978,125194,13510,30354,1241,3290935,220729,29975,636424,...,4464663,238079,96368,4,212998,4112,303658,6538,92409,38554
2021-05-16,63484,132015,125311,13510,30637,1241,3307285,220860,29978,637097,...,4466589,240512,96670,4,215301,4212,303827,6543,92436,38560


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,2730,2427,3360,127,651,32,69853,4301,910,10455,...,127930,3369,667,1,2366,35,3401,1285,1260,1582
2021-05-15,2733,2429,3366,127,655,33,70253,4314,910,10470,...,127937,3419,667,1,2366,36,3423,1285,1260,1582
2021-05-16,2742,2432,3374,127,659,33,70522,4323,910,10474,...,127941,3459,668,1,2396,37,3428,1289,1260,1582


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,54634,122105,87137,13155,25650,1179,2913144,207464,23490,612523,...,15214,204051,91507,3,196759,2657,288891,3008,90699,36308
2021-05-15,54663,123081,87251,13155,25703,1179,2933946,208100,23493,614803,...,15254,206670,91831,3,196759,2668,291191,3014,90777,36318
2021-05-16,54686,124312,87359,13155,25715,1179,2952599,208454,23494,616148,...,15258,209338,92111,3,199202,2668,292523,3023,90816,36329


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6918,6918,6920,6920,6926,6928,6938,6971,7001,7005
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1419,1425,1425,1425,1425,1427,1429,1430,1432,1433
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9419,9420,9424,9428,9427,9437,9441,9449,9457,9460
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2998,3002,3002,3002,3003,3004,3007,3022,3026,3030
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15856,15873,15884,15886,15882,15887,15892,15946,15987,15993


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-14,537813,69464,0,872022,338485,3767052,530098,344977,106715,49,...,857055,2927195,402005,23792,3267,669219,421757,157923,668805,59079
2021-05-15,539829,69464,0,872496,338687,3768087,531070,344977,106873,49,...,857055,2928389,402325,23847,3267,669904,422665,158230,669427,59079
2021-05-16,540083,69464,0,872978,338782,3768737,531700,344977,107007,49,...,857055,2929050,402567,23883,3267,670184,422665,158468,669668,59079


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-05-14        6971   21290    2319  2630   6750    1230   2178   14556   
2021-05-15        7001   21392    2320  2645   6771    1232   2190   14569   
2021-05-16        7005   21411    2320  2647   6773    1232   2191   14574   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-14         3581     1843  ...    2758    630     3179      790   
2021-05-15         3590     1846  ...    2758    630     3179      790   
2021-05-16         3596     1848  ...    2758    630     3179      790   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-14           4353  3765  2239          0      918    651  
2021-05-15           4353  3765  2239          0      918    651  
2021-05-16           4353  3765  2239          0      918    651  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-14,11024,353,0,17447,5790,62633,6432,8173,1651,0,...,12312,50903,2256,252,27,10991,5622,2756,7707,712
2021-05-15,11038,353,0,17459,5793,62656,6442,8173,1651,0,...,12312,50940,2258,252,27,11008,5622,2757,7712,712
2021-05-16,11038,353,0,17459,5794,62690,6450,8173,1651,0,...,12312,50941,2258,252,27,11018,5622,2761,7712,712


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-05-14         108     309      56   64    139      41     69     319   
2021-05-15         108     310      56   64    139      42     69     319   
2021-05-16         108     310      56   64    139      42     69     319   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-14          123       45  ...      30     11       31        7   
2021-05-15          123       45  ...      30     11       31        7   
2021-05-16          123       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-14             38     9    13          0       26      5  
2021-05-15             38     9    13          0       26      5  
2021-05-16             38     9    13          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,0.004917,0.000372,0.001361,0.00297,0.011281,0.000000,0.008440,0.001044,0.000234,0.001397,...,0.000493,0.014252,0.003037,0.0,0.005476,0.020321,0.000000,0.002459,0.001019,0.001143
2021-05-15,0.000900,0.000296,0.001079,0.00000,0.010789,0.000806,0.006567,0.001279,0.000367,0.001013,...,0.000462,0.012215,0.002611,0.0,0.000000,0.077568,0.001279,0.002300,0.000574,0.000493
2021-05-16,0.001135,0.000280,0.000935,0.00000,0.009323,0.000000,0.004968,0.000593,0.000100,0.001057,...,0.000431,0.010219,0.003134,0.0,0.010812,0.024319,0.000557,0.000765,0.000292,0.000156


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,0.003308,0.000412,0.001490,0.0,0.003082,0.00000,0.008649,0.002330,0.0,0.001053,...,0.000141,0.015371,0.000000,0.0,0.005952,0.000000,0.000000,0.005477,0.0,0.0
2021-05-15,0.001099,0.000824,0.001786,0.0,0.006144,0.03125,0.005726,0.003023,0.0,0.001435,...,0.000055,0.014841,0.000000,0.0,0.000000,0.028571,0.006469,0.000000,0.0,0.0
2021-05-16,0.003293,0.001235,0.002377,0.0,0.006107,0.00000,0.003829,0.002086,0.0,0.000382,...,0.000031,0.011699,0.001499,0.0,0.012680,0.027778,0.001461,0.003113,0.0,0.0


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,0.000275,0.008116,0.001540,0.003892,0.000819,0.0,0.007508,0.003444,0.000255,0.002076,...,0.000855,0.013329,0.002256,0.0,0.007058,0.00000,0.000000,0.001665,0.000706,0.000606
2021-05-15,0.000531,0.007993,0.001308,0.000000,0.002066,0.0,0.007141,0.003066,0.000128,0.003722,...,0.002629,0.012835,0.003541,0.0,0.000000,0.00414,0.007961,0.001995,0.000860,0.000275
2021-05-16,0.000421,0.010002,0.001238,0.000000,0.000467,0.0,0.006358,0.001701,0.000043,0.002188,...,0.000262,0.012909,0.003049,0.0,0.012416,0.00000,0.004574,0.002986,0.000430,0.000303


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-14,0.009229,0.001629,NaN,0.000980,0.000739,0.000703,0.002961,0.001059,0.001267,0.0,...,0.000732,0.000830,0.000837,0.002866,0.00184,0.000737,0.005663,0.001884,0.000893,0.001407
2021-05-15,0.003749,0.000000,NaN,0.000544,0.000597,0.000275,0.001834,0.000000,0.001481,0.0,...,0.000000,0.000408,0.000796,0.002312,0.00000,0.001024,0.002153,0.001944,0.000930,0.000000
2021-05-16,0.000471,0.000000,NaN,0.000552,0.000280,0.000173,0.001186,0.000000,0.001254,0.0,...,0.000000,0.000226,0.000602,0.001510,0.00000,0.000418,0.000000,0.001504,0.000360,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-14      0.004756  0.004672  0.000863  0.005736  0.008366  0.001629   
2021-05-15      0.004304  0.004791  0.000431  0.005703  0.003111  0.001626   
2021-05-16      0.000571  0.000888  0.000000  0.000756  0.000295  0.000000   

Province_State                                          ...  Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...     Park   
2021-05-14      0.006470  0.002341  0.000838 -0.002166  ...  0.00218   
2021-05-15      0.005510  0.000893  0.002513  0.001628  ...  0.00000   
2021-05-16      0.000457  0.000343  0.001671  0.001083  ...  0.00000   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2021-05-14     -0.001585  0.000315      0.0   0.003689  0.000797  0.000894   
2021-05-15      0.000000  0.000000      0.0   0.000000  0.000000  0.000000   
2021-05-16      0.000000  0.000000      0.0   0.000000  0.000000  0.000000   

Province_State                             
Admin2         Unassigned Washakie Weston  
2021-05-14            NaN      0.0    0.0  
2021-05-15            NaN      0.0    0.0  
2021-05-16            NaN      0.0    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-14,0.002091,0.0,NaN,0.000516,0.001210,0.000959,0.001401,0.000612,0.00182,NaN,...,0.000569,0.001239,0.000443,0.0,0.0,0.002737,-0.000711,0.001818,0.000130,0.0
2021-05-15,0.001270,0.0,NaN,0.000688,0.000518,0.000367,0.001555,0.000000,0.00000,NaN,...,0.000000,0.000727,0.000887,0.0,0.0,0.001547,0.000000,0.000363,0.000649,0.0
2021-05-16,0.000000,0.0,NaN,0.000000,0.000173,0.000543,0.001242,0.000000,0.00000,NaN,...,0.000000,0.000020,0.000000,0.0,0.0,0.000908,0.000000,0.001451,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                   \
Admin2         Autauga   Baldwin Barbour Bibb    Blount  Bullock Butler   
2021-05-14         0.0  0.000000     0.0  0.0  0.014599  0.00000    0.0   
2021-05-15         0.0  0.003236     0.0  0.0  0.000000  0.02439    0.0   
2021-05-16         0.0  0.000000     0.0  0.0  0.000000  0.00000    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2021-05-14      0.009494      0.0      0.0  ...     0.0    0.0      0.0   
2021-05-15      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2021-05-16      0.000000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-14          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-15          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-16          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,0.004945,0.000358,0.001493,0.00184,0.010253,0.000392,0.008088,0.001150,0.000207,0.001487,...,0.000526,0.014020,0.003441,1.986821e-08,0.004963,0.022172,0.000410,0.001864,0.000778,0.000837
2021-05-15,0.002922,0.000327,0.001286,0.00092,0.010521,0.000599,0.007327,0.001214,0.000287,0.001250,...,0.000494,0.013118,0.003026,9.934107e-09,0.002482,0.049870,0.000845,0.002082,0.000676,0.000665
2021-05-16,0.002029,0.000304,0.001110,0.00046,0.009922,0.000300,0.006148,0.000904,0.000194,0.001154,...,0.000463,0.011668,0.003080,4.967054e-09,0.006647,0.037095,0.000701,0.001423,0.000484,0.000410


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,0.003019,0.000849,0.001663,2.018561e-06,0.004878,6.153567e-08,0.007688,0.003053,1.280697e-13,0.001291,...,0.000115,0.016182,0.000991,0.0,0.006307,1.373851e-78,0.000851,0.003236,0.000357,0.000599
2021-05-15,0.002059,0.000837,0.001724,1.009280e-06,0.005511,1.562503e-02,0.006707,0.003038,6.403483e-14,0.001363,...,0.000085,0.015512,0.000495,0.0,0.003154,1.428571e-02,0.003660,0.001618,0.000178,0.000299
2021-05-16,0.002676,0.001036,0.002050,5.046402e-07,0.005809,7.812515e-03,0.005268,0.002562,3.201742e-14,0.000873,...,0.000058,0.013606,0.000997,0.0,0.007917,2.103175e-02,0.002560,0.002366,0.000089,0.000150


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-14,0.000901,0.008349,0.001619,0.002645,0.006071,0.00296,0.007159,0.003123,0.000341,0.002510,...,0.001676,0.012464,0.002917,4.768372e-07,0.006589,0.003173,0.000817,0.001061,0.000641,0.000730
2021-05-15,0.000716,0.008171,0.001464,0.001323,0.004069,0.00148,0.007150,0.003094,0.000235,0.003116,...,0.002153,0.012649,0.003229,2.384186e-07,0.003295,0.003657,0.004389,0.001528,0.000751,0.000503
2021-05-16,0.000568,0.009086,0.001351,0.000661,0.002268,0.00074,0.006754,0.002398,0.000139,0.002652,...,0.001207,0.012779,0.003139,1.192093e-07,0.007855,0.001828,0.004482,0.002257,0.000590,0.000403


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-14,0.005296,0.001455,NaN,0.000809,0.000660,0.000530,0.002924,0.001102,0.001200,7.857854e-129,...,0.000814,0.000813,0.000899,0.002758,0.002564,0.000794,0.004341,0.002086,0.000873,0.001426
2021-05-15,0.004522,0.000727,NaN,0.000676,0.000628,0.000403,0.002379,0.000551,0.001340,3.928927e-129,...,0.000407,0.000611,0.000848,0.002535,0.001282,0.000909,0.003247,0.002015,0.000902,0.000713
2021-05-16,0.002496,0.000364,NaN,0.000614,0.000454,0.000288,0.001783,0.000275,0.001297,1.964464e-129,...,0.000204,0.000418,0.000725,0.002022,0.000641,0.000663,0.001624,0.001760,0.000631,0.000357


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-14      0.002837  0.002763  0.001035  0.003365  0.004932  0.000714   
2021-05-15      0.003570  0.003777  0.000733  0.004534  0.004022  0.001170   
2021-05-16      0.002071  0.002333  0.000367  0.002645  0.002159  0.000585   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-05-14      0.003499  0.001374  0.000895 -0.001608  ...  0.002284   
2021-05-15      0.004504  0.001134  0.001704  0.000010  ...  0.001142   
2021-05-16      0.002480  0.000738  0.001688  0.000547  ...  0.000571   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-05-14      0.001105  0.001009  0.000564   0.003217  0.000686  0.001166   
2021-05-15      0.000553  0.000505  0.000282   0.001608  0.000343  0.000583   
2021-05-16      0.000276  0.000252  0.000141   0.000804  0.000171  0.000292   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-05-14           -1.0  0.001340  4.112810e-07  
2021-05-15           -1.0  0.000670  2.056405e-07  
2021-05-16           -1.0  0.000335  1.028203e-07  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-14,0.001333,0.001077,NaN,0.000471,0.001105,0.001164,0.002286,0.000684,0.001764,NaN,...,0.000651,0.001154,0.001957,0.001009,5.731216e-10,0.002308,0.000867,0.002010,0.001181,0.000178
2021-05-15,0.001301,0.000538,NaN,0.000580,0.000812,0.000765,0.001920,0.000342,0.000882,NaN,...,0.000325,0.000940,0.001422,0.000505,2.865608e-10,0.001928,0.000434,0.001186,0.000915,0.000089
2021-05-16,0.000651,0.000269,NaN,0.000290,0.000492,0.000654,0.001581,0.000171,0.000441,NaN,...,0.000163,0.000480,0.000711,0.000252,1.432804e-10,0.001418,0.000217,0.001319,0.000457,0.000045


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-14      0.000009  0.000435 -0.002158  0.000062  0.007335  0.000002   
2021-05-15      0.000005  0.001835 -0.001079  0.000031  0.003668  0.012196   
2021-05-16      0.000002  0.000918 -0.000540  0.000016  0.001834  0.006098   

Province_State                                                      ...  \
Admin2                Butler   Calhoun      Chambers      Cherokee  ...   
2021-05-14      7.074968e-09  0.005559  7.515225e-07  6.780024e-10  ...   
2021-05-15      3.537484e-09  0.002779  3.757613e-07  3.390012e-10  ...   
2021-05-16      1.768742e-09  0.001390  1.878806e-07  1.695006e-10  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-05-14      3.828859e-18  1.554119e-34  2.891794e-20  1.270391e-40   
2021-05-15      1.914430e-18  7.770597e-35  1.445897e-20  6.351956e-41   
2021-05-16      9.572148e-19  3.885298e-35  7.229485e-21  3.175978e-41   

Province_State                                                              \
Admin2         Sweetwater         Teton     Uinta Unassigned      Washakie   
2021-05-14       0.000013  4.055327e-28  0.000041  -0.501961  7.893154e-33   
2021-05-15       0.000007  2.027664e-28  0.000020  -0.501961  3.946577e-33   
2021-05-16       0.000003  1.013832e-28  0.000010  -0.501961  1.973288e-33   

Province_State                
Admin2                Weston  
2021-05-14      1.676329e-31  
2021-05-15      8.381647e-32  
2021-05-16      4.190824e-32  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
